In [28]:
import pandas as pd
import os
import pathlib as Path
import matplotlib.pyplot as plt
import glob
import ast
import re
pd.set_option('display.max_rows', None)

In [19]:
path = os.path.join('..', 'results')  # This navigates from notebooks to results folder

# Get all CSV files in the folder
all_files = glob.glob(os.path.join(path, "*.csv"))

# Create an empty list to store individual dataframes
dfs = []

# Read each CSV file and append to the list
for file in all_files:
    df_temp = pd.read_csv(file)
    # Optional: Add a column to track which file the data came from
    df_temp['source_file'] = os.path.basename(file)
    dfs.append(df_temp)

# Combine all dataframes into one
df = pd.concat(dfs, ignore_index=True)

In [18]:
#When there is a title for somebody whose status is not recognised.
conditions = (
    # person_id is not empty
    df['person_id'].notna() & (df['person_id'] != 'N/A') &
    
    # social_status_job is empty
    ((df['social_status_job'].isna()) | (df['social_status_job'] == 'N/A')) &
    
    # titles is not empty and not an empty list
    df['titles'].notna() & (df['titles'] != 'N/A') & (df['titles'] != '[]')
)
filtered_df = df[conditions]
filtered_df.head()

,person_id,name,gender,religion_ethnicity,social_status_job,role_in_case,titles,date,calendar,place_name,place_type,legal_case_type,case_result,case_result_type,row_index,case_unique_id,source_file
0,beşiktaş mahkemesi_2_hüküm no: 1_1,Mehmed Bey b. Abdullah,Man,Muslim,NaN,Borrower,['Bey'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,9516,beşiktaş mahkemesi_2_hüküm no: 1,Beşiktaş Mahkemesi_2.csv
1,beşiktaş mahkemesi_2_hüküm no: 1_2,Keyvan Kethüdâ b. Abdurrahman,Man,Muslim,NaN,Buyer,['Kethüdâ'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,9516,beşiktaş mahkemesi_2_hüküm no: 1,Beşiktaş Mahkemesi_2.csv
5,beşiktaş mahkemesi_2_hüküm no: 1_6,Mehmed Ağa b. Pîr Ağa,Man,Muslim,NaN,Witness,['Ağa'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,9516,beşiktaş mahkemesi_2_hüküm no: 1,Beşiktaş Mahkemesi_2.csv
6,beşiktaş mahkemesi_2_hüküm no: 1_7,Şeref Hâce,Man,Muslim,NaN,Witness,['Hâce'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,9516,beşiktaş mahkemesi_2_hüküm no: 1,Beşiktaş Mahkemesi_2.csv
20,beşiktaş mahkemesi_2_hüküm no: 2_5,Nasuh Bey b. Abdullah,Man,Muslim,NaN,Witness,['fahrü’l-akrân'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,9517,beşiktaş mahkemesi_2_hüküm no: 2,Beşiktaş Mahkemesi_2.csv


In [32]:
filtered_df = df[conditions].copy()

# First convert the string representation of lists to actual lists
filtered_df.loc[:, 'titles_list'] = filtered_df['titles'].str.strip('[]').str.split(',')

# Clean up each title in the lists
filtered_df.loc[:, 'titles_list'] = filtered_df['titles_list'].apply(
    lambda titles: [t.strip().strip("'\"") for t in titles if t.strip().strip("'\"") and t.strip().strip("'\"") != 'N/A']
)

# Explode the lists to get one title per row
titles_series = filtered_df['titles_list'].explode()

# Get unique values and sort them
unique_titles = titles_series.unique()
unique_titles_sorted = pd.Series(unique_titles).sort_values().reset_index(drop=True)

# Get value counts for frequency analysis
title_counts = titles_series.value_counts().reset_index()
title_counts.columns = ['title', 'count']

print("\nTop titles by frequency:")
print(title_counts.head(500))


Top titles by frequency:
                                                title  count
0                                              Çelebi  25907
1                                              Efendi  11349
2                                                 Bey   9405
3                                                 Ağa   7474
4                                              el-Hâc   7122
5                                                Beşe   4961
6                                               Çavuş   2430
7                                                Hacı   1700
8                                           es-Seyyid   1649
9                                              Halîfe   1459
10                                               Reis   1448
11                                              Hâtun   1201
12                                              Hatun   1035
13                                             Seyyid    878
14                                             El-Hâc    82

In [ ]:
title_to_job_mapping = {
    'Çelebi': 'undesignated middle class',
    'Efendi': 'undesignated middle class',
    'Bey': 'undesignated middle class',
    'Ağa': 'military',
    'el-Hâc': 'religious',
    'Beşe': 'military',
    'Çavuş': 'military',
    'Hacı': 'religious',
    'es-Seyyid': 'religious',
    'Halîfe': 'undesignated middle class',
    'Reis': 'undesignated middle class',
    'Hâtun': 'undesignated middle class',   
    'Hatun': 'undesignated middle class',   
    'Seyyid': 'religious',    
    'El-Hâc': 'religious',   
    'el-İmâm': 'religious',    
    'el-Muhzır': 'administrative',    
    'el-muhzır': 'administrative',    
    'zimmî': 'undesignated middle class',    
    'Mevlânâ': 'administrative',    
    'er-râcil': 'undesignated middle class',    
    'Baba': 'sufi',    
    'Dede': 'sufi',    
    'el-Müezzin': 'religious',    
    'el-Mübâşir': 'administrative',    
    'el-imâm': 'religious',    
    'er-Râcil': 'undesignated middle class',    
    'el-mübâşir': 'administrative',  
    'Kethüdâ': 'artisanal',    
    'Halife': 'undesignated middle class',    
    'Fahrü’l-akrân': 'undesignated middle class',    
    'Derviş': 'sufi',    
    'el-Cündî': 'military',    
    'Es-Seyyid': 'religious',    
    'Şeyh': 'sufi',    
    'el-müezzin': 'religious',    
    'Fahrü’l-muhadderât': 'religious',    
    'Zimmi': 'undesignated middle class',    
    'Hanım': 'undesignated middle class',    
    'Zimmî': 'undesignated middle class',    
    'Reisü’l-muhzırîn': 'administrative',    
    'el-kâtib': 'administrative',    
    'el-cündî': 'military',    
    'Odabaşı': 'administrative',    
    'Muhzır': 'administrative',    
    'Papa': 'religious',    
    'fahrü’l-muhadderât': 'administrative',    
    'çukadâr': 'military',    
    'Paşa': 'administrative',    
    'İmâm': 'religious',    
    'Üstâd': 'religious',
    'İmâm': 'religious',    
    'Üstâd': 'religious',    
    'Müezzin': 'religious',    
    'fahrü’l-akrân': 'undesignated middle class',   
    'el-Kâtib': 'administrative',     
    'el-bevvâb': 'administrative',     
    'eş-Şeyh': 'sufi',    
    'Hoca': 'religious',    
    'Şerife': 'undesignated middle class',    
    'Fahrü’l-a‘yân': 'undesignated middle class',     
    'Subaşı': 'military',    
    'Kapudan': 'military',     
    'Fahrü’l-kuzât': 'administrative',     
    'Mütevellî': 'administrative',      
    'Hâfız': 'religious',     
    'el-Hac': 'religious',     
    'Sultan': 'administrative',      
    'Çukadâr': 'military',      
    'Kapudân': 'military',      
    'el-cündi': 'military',      
    'el-Hatîb': 'administrative',      
    'Elhâc': 'religious',     
    'el-Bevvâbü’s-sultânî': 'administrative',      
    'el-mütevellî': 'administrative',      
    'Sofu': 'sufi',     
    'el-Kayyım': 'administrative',     
    'Hâce': 'religious',     
    'el-bevvâbü’s-sultânî': 'administrative',     
    'Molla': 'religious',     
    'Kethudâ': 'military',     
    'Papas': 'religious',     
    'hâtun': 'undesignated middle class',     
    'Fahrü’l-eşbâh': 'undesignated middle class',     
    'el-müteferrika': 'administrative',     
    'ases': 'military',     
    'el-Bevvâb': 'administrative',     
    'el-mülâzım': 'administrative',     
    'Fahrü’l-eimme': 'administrative',     
    'el-Mütevellî': 'administrative',     
    'Kâtib': 'administrative',     
    'er-Reis': 'military',     
    'Çorbacı': 'military',     
    'fahrü’l-a‘yân': 'undesignated middle class',     
    'el-hatîb': 'administrative',     
    'El-hâc': 'religious',     
    'Şuhûdü’l-hâl': 'undesignated middle class',     
    'Hâcı': 'religious',     
    'el-kādî': 'administrative',     
    'Umdetü’l-a‘yân': 'undesignated middle class',     
    'Pîr': 'sufi',     
    'umdetü’l-muhadderât': 'undesignated middle class',     
    'Ases': 'military',     
    'zahrü’l-muvakkarât': 'administrative',     
    'Bölükbaşı': 'military',     
    'el-Yesârî': 'administrative',     
    'Ser-bölük': 'military',     
    'Bevvâb': 'administrative',     
    'Hâc': 'religious',     
    'el-Kadı': 'administrative',     
    'Ser-muhzırân': 'administrative',     
    'er-Reisü’s-sultânî': 'undesignated middle class',     
    'fahrü’l-eşbâh': 'undesignated middle class',     
    'Er-râcil': 'undesignated middle class',     
    'Kapıcı': 'military',     
    'Mühtedî': 'undesignated middle class',     
    'el-müsta‘id': 'administrative',     
    'el-câbî': 'administrative',     
    'Râcil': 'undesignated middle class',     
    'el-kayyım': 'administrative',     
    'Usta': 'artisanal',     
    'tâcü’l-mestûrât': 'undesignated middle class',     
    'Mevlana': 'religious',     
    'Mühtedi': 'undesignated middle class',     
    'Bevvâb-ı sultânî': 'administrative',     
    'Kethüdâ-yı muhzırân': 'administrative',     
    'el-fakīr': 'undesignated middle class',     
    'İmam': 'religious',     
    'aliyyetü’z-zât': 'undesignated middle class',     
    'Fahrü’l-emâsil ve’l-akrân': 'undesignated middle class',     
    'el-yesârî': 'artisanal',     
    'Fahrü’l-cüyûş': 'military',     
    'ez-za‘îm': 'administrative',     
    'el-Muhtesib': 'administrative',     
    'Eş-Şeyh': 'sufi',     
    'kethüdâ-yı muhzırân': 'administrative',     
    'Zahrü’l-muvakkarât': 'administrative',     
    'Reîs': 'military',     
    'mevlânâ': 'religious',     
    'el-Müteferrika': 'administrative',     
    'Reis-i sultânî': 'undesignated middle class',     
    'muhzır': 'administrative',     
    'el-müvellâ': 'undesignated middle class',     
    'Silahdâr': 'military',     
    'bârikatü’l-envârı’l-melekiyye': 'undesignated middle class',     
    'Zahrü’l-akrân': 'undesignated middle class',     
    'Yazıcı': 'administrative',     
    'Hafız': 'religious',     
    'El-Hac': 'religious',     
    'Kadın': 'undesignated middle class',     
    'El-Muhzır': 'administrative',     
    'nâdiretü’l-edvârı’l-felekiyye': 'undesignated middle class',     
    'Muhzırbaşı': 'military',

    151                                           kethüdâ     16
152                                         Yiğitbaşı     16
153                        Kethüdâ-yı Hazret-i Efendi     16
154                              Sultanü’l-muhadderât     16
155                                         Nasrâniye     16
156                                  safiyetü’s-sıfât': 'undesignated middle class',     16
157                                         el-kayyum     16
158                                   Reisü’s-sultânî     16
159                                   Tâcü’l-mestûrât': 'undesignated middle class',     16
160                                 Muharrirü’l-hurûf': 'undesignated middle class',     16
161                                             Solak     16
162                                       el-mukayyid     15
163                                         el-Münâdî     15
164                                     fahrü’l-eimme': 'undesignated middle class',     15
166                                          Şeyhzâde     15
167                                     fahrü’l-kuzât     15
168                                         ez-zevvâk     14
169                                           odabaşı     14
170                                             Za‘îm     14
171                                         El-muhzır     14
172                                         Çavuşbaşı     13
173                                              Zaîm     13
174                    Sâhibetü’l-hayrât ve’l-hasenât     13
175                              zübdetü’l-muvakkarât     13
176                                    Hâfızü’l-kitâb     13
177                                    Fahrü’l-emâsil     13
178                sadefü dürretü’l-hilâfeti’l-bâhire     13
179                                      ser-muhzırân     13
180                                           Çukadar     13
181                                    umdetü’l-a‘yân     13
182                                 sahibetü’l-hayrât     12
183                                             Monla     12
184                                râgıbetü’s-sadakāt     12
185                         fahrü’l-emâsil ve’l-akrân     12
186                                               Pir     12
187                                            Kayyım     12
188                                         es-silâhî     12
189                                            Merhûm     12
190                                           el-Kādî     12
191                                             solak     12
192                                 sâhibetü’l-hayrât     12
193                                    Kâtibü’l-hurûf     12
194                                         el-kayyûm     12
195                                             Hâcce     11
196                                        min huddâm     11
197                                           el-kadı     11
198                                          Türbedâr     11
199                                        el-Mülâzım     11
200                                     ve’l-meberrât     11
201                                            Korucu     11
202                                  bevvâb-ı sultânî     11
203                                         Mutallaka     11
204                                      ve’l-hasenât     11
205                                         Es-seyyid     11
206                                       el-müste‘id     11
207                               Umdetü’l-muhadderât     11
208  matla‘u ‘izzeti’s-saltanati’s-seniyyeti’l-kāhire     11
209                                 Sâhibetü’l-hayrât     11
210                                             Papaz     11
211                                            Haseki     11
212                                         Mütevelli     11
213                                              Câbî     10
214                                       el-Müsta‘id     10
215                                   Udûl-i Müslimîn     10
216                                  reisü’l-muhzırîn     10
217                                             Cündî     10
218                                          El-fakīr     10
219                                             Râhib     10
220                                           El-İmâm     10
221                                            merhûm     10
222                              kıdvetü’l-muhadderât     10
223                            Seyyidetü’l-muhadderât     10
224                                          Er-Râcil     10
225                                         es-sarrâc     10
226                                         bölükbaşı     10
227                                      zâtü’l-aliyy     10
228                                           Kethüda     10
229                                           ed-Dâ‘î      9
230                                             Fakīh      9
231                                          ez-Za‘îm      9
232                                           çukadar      9
233                                    kâtibü’l-hurûf      9
234                                          Teberdâr      9
235                                         Hazînedâr      9
236                                        El-Müezzin      9
237                                           el-Emîn      9
238                                  Bevvâb-ı Sultânî      9
239                                       el-mu‘allim      9
240                                  Reisü’l-muhzirîn      9
241                                             Topçu      9
242                                          eş-şehîr      9
243                                      ve’s-sa‘âdât      9
244                                         es-Sarrâc      9
245                                            Sagīre      9
246                        Fahrü’l-emâcid ve’l-ekârim      9
247                                          Bostancı      9
248                                  zevvâk-ı sultânî      9
249                                             Şeyhî      9
250                                        muhzırbaşı      9
251                                          El-Cündî      8
252                                    Reisü’l-muhzır      8
253                            seyyidetü’l-muhadderât      8
254                                    Umdetü’l-kuzât      8
255                                           Mübâşir      8
256                                       el-mü’ezzin      8
257                                        el-Bostânî      8
258                                         el-havâle      8
259                                    Sâhibü’l-kitâb      8
260                                             Hâtûn      8
261                                         El-Bevvâb      8
262                                             Fakih      8
263                                            Serdâr      8
264                                        El-müezzin      8
265                                    fahrü’l-emâsil      8
266                                          en-Nâzır      8
267                                           Saraylı      8
268                                         el-kassâm      8
269                                 muharrirü’l-hurûf      8
270                                           zimmîye      8
271                                   udûl-i müslimîn      8
272                                           El-imâm      7
273                                              Kara      7
274                                             râcil      7
275                                              Yaya      7
276                              bürhânü’l-muvakkarât      7
277                               el-Bevvâb-ı sultânî      7
278                                        el-Hammâmî      7
279                              es-Sarrâcü’s-sultânî      7
280                                         ser-bölük      7
281                                              Kadı      7
282                                          el-Hâfız      7
283                                             Hatîb      7
284                                          Asesbaşı      7
285                                          El-cündî      7
286                                      Peyk-i hâssa      7
287                                     zahrü’l-cüyûş      7
288                    sâhibetü’l-hayrât ve’l-hasenât      7
289                                      Vekîl-i harc      7
290                                             Keşiş      7
291                  râgıbetü’s-sadakāt ve’l-meberrât      7
292                                          Mü’ezzin      7
293                                         el-münâdî      7
294                                        El-Mübâşir      7
295                                             cündî      7
296                                             Haham      7
297                         zâtü’l-aliyy ve’s-sa‘âdât      7
298                                Fahrü’l-müderrisîn      7
299                                     Zahrü’l-a‘yân      7
300                 Sultanü’l-muhadderâti’l-mu‘azzama      6
}